<a href="https://colab.research.google.com/github/MichaelAshton/piano_transcription/blob/master/music_translation_inference_teslav100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvidia-smi

Sun Aug 11 05:23:51 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    16W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
!git clone https://github.com/facebookresearch/music-translation.git
%cd music-translation
!pip install -r requirements.txt

fatal: destination path 'music-translation' already exists and is not an empty directory.
/content/music-translation


In [5]:
%cd src/nv-wavenet
!python setup.py install
%cd ../..

[Errno 2] No such file or directory: 'src/nv-wavenet'
/content/music-translation/src/nv-wavenet
running install
running bdist_egg
running egg_info
writing nv_wavenet_ext.egg-info/PKG-INFO
writing dependency_links to nv_wavenet_ext.egg-info/dependency_links.txt
writing top-level names to nv_wavenet_ext.egg-info/top_level.txt
writing manifest file 'nv_wavenet_ext.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_ext
creating build/bdist.linux-x86_64/egg
copying build/lib.linux-x86_64-3.6/nv_wavenet_ext.cpython-36m-x86_64-linux-gnu.so -> build/bdist.linux-x86_64/egg
creating stub loader for nv_wavenet_ext.cpython-36m-x86_64-linux-gnu.so
byte-compiling build/bdist.linux-x86_64/egg/nv_wavenet_ext.py to nv_wavenet_ext.cpython-36.pyc
creating build/bdist.linux-x86_64/egg/EGG-INFO
copying nv_wavenet_ext.egg-info/PKG-INFO -> build/bdist.linux-x86_64/egg/EGG-INFO
copying nv_wavenet_ext.egg-info/SOURCES.txt -> build/bdist.linux-x86_64/

In [9]:
!mkdir checkpoints
%cd checkpoints

/content/music-translation/checkpoints


In [10]:
!wget https://dl.fbaipublicfiles.com/music-translation/pretrained_musicnet.zip
!unzip pretrained_musicnet.zip

--2019-08-11 05:09:13--  https://dl.fbaipublicfiles.com/music-translation/pretrained_musicnet.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.20.22.166, 104.20.6.166, 2606:4700:10::6814:16a6, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.20.22.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 380659262 (363M) [application/zip]
Saving to: ‘pretrained_musicnet.zip’

pretrained_musicnet 100%[===================>] 363.02M  34.1MB/s    in 11s     

2019-08-11 05:09:24 (32.6 MB/s) - ‘pretrained_musicnet.zip’ saved [380659262/380659262]

Archive:  pretrained_musicnet.zip
   creating: pretrained_musicnet/
  inflating: pretrained_musicnet/args.pth  
  inflating: pretrained_musicnet/bestmodel_3.pth  
  inflating: pretrained_musicnet/bestmodel_2.pth  
  inflating: pretrained_musicnet/bestmodel_1.pth  
  inflating: pretrained_musicnet/bestmodel_0.pth  
  inflating: pretrained_musicnet/bestmodel_5.pth  
  inflating: pretrained

In [13]:
!ls pretrained_musicnet

args.pth	 bestmodel_3.pth  lastmodel_1.pth  lastmodel_5.pth
bestmodel_0.pth  bestmodel_4.pth  lastmodel_2.pth
bestmodel_1.pth  bestmodel_5.pth  lastmodel_3.pth
bestmodel_2.pth  lastmodel_0.pth  lastmodel_4.pth


In [14]:
%cd ..

/content/music-translation


In [3]:
%cd music-translation

/content/music-translation


In [5]:
!ls

checkpoints	    img        README.md	 sample.sh	train.sh
CODE_OF_CONDUCT.md  LICENSE    requirements.txt  src
CONTRIBUTING.md     notebooks  sample_py.sh	 train_dist.sh


In [0]:
from pathlib import Path
import librosa
import torch
from argparse import ArgumentParser
import matplotlib
import h5py
import tqdm
from IPython.display import Audio, display

import sys
sys.path += ['/content/music-translation/src']

import utils
import wavenet_models
from utils import save_audio
from wavenet import WaveNet
from wavenet_generator import WavenetGenerator
from nv_wavenet_generator import NVWavenetGenerator

In [0]:
# Inference params
# checkpoint = Path('/content/music-translation/checkpoints/pretrained_musicnet/lastmodel')
# decoders = [0, 1, 2, 3, 4, 5]
checkpoint = Path('/content/music-translation/checkpoints/pretrained_musicnet/bestmodel')
decoders = [1]
batch_size = 1
rate = 16000
split_size = 20
file_paths = [Path('/content/test.wav')]

In [0]:
def disp(x, decoder_ix):
    wav = utils.inv_mu_law(x.cpu().numpy())
    print(f'Decoder: {decoder_ix}')
    print(f'X min: {x.min()}, max: {x.max()}')

    display(Audio(wav.squeeze(), rate=rate))
        
def extract_id(path):
    decoder_id = str(path)[:-4].split('_')[-1]
    return int(decoder_id)

In [4]:
print('Starting')
matplotlib.use('agg')

checkpoints = checkpoint.parent.glob(checkpoint.name + '_*.pth')
checkpoints = [c for c in checkpoints if extract_id(c) in decoders]
assert len(checkpoints) >= 1, "No checkpoints found."

model_args = torch.load(checkpoint.parent / 'args.pth')[0]
encoder = wavenet_models.Encoder(model_args)
encoder.load_state_dict(torch.load(checkpoints[0])['encoder_state'])
encoder.eval()
encoder = encoder.cuda()

decoders = []
decoder_ids = []
for checkpoint in checkpoints:
    decoder = WaveNet(model_args)
    decoder.load_state_dict(torch.load(checkpoint)['decoder_state'])
    decoder.eval()
    decoder = decoder.cuda()
    decoder = NVWavenetGenerator(decoder, rate * (split_size // 20), batch_size, 2)

    decoders += [decoder]
    decoder_ids += [extract_id(checkpoint)]

Starting


/content/music-translation/src/wavenet.py:169: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].
  inp = torch.range(0, 255) / 255 - 0.5


In [5]:
!nvidia-smi

Sun Aug 11 06:55:12 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P0    28W /  70W |   1449MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [6]:
xs = []

for file_path in file_paths:
    data, rate = librosa.load(file_path, sr=16000)
    assert rate == 16000
    data = utils.mu_law(data)
    xs.append(torch.tensor(data).unsqueeze(0).float().cuda())

xs = torch.stack(xs).contiguous()
print(f'xs size: {xs.size()}')

xs size: torch.Size([1, 1, 80000])


In [7]:
yy = {}
with torch.no_grad():
    zz = []
    for xs_batch in torch.split(xs, batch_size):
        zz += [encoder(xs_batch)]
    zz = torch.cat(zz, dim=0)

    with utils.timeit("Generation timer"):
        for i, decoder_id in enumerate(decoder_ids):
            print(f'decoder id : {decoder_id}')
            yy[decoder_id] = []
            decoder = decoders[i]
            for zz_batch in torch.split(zz, batch_size):
                print(zz_batch.shape)
                splits = torch.split(zz_batch, split_size, -1)
                audio_data = []
                decoder.reset()
                for cond in tqdm.tqdm_notebook(splits):
                    audio_data += [decoder.generate(cond).cpu()]
                audio_data = torch.cat(audio_data, -1)
                yy[decoder_id] += [audio_data]
            yy[decoder_id] = torch.cat(yy[decoder_id], dim=0)

decoder id : 1
torch.Size([1, 64, 100])



Generation timer took 18230.924367904663 ms


In [8]:
for decoder_ix, decoder_result in yy.items():
    for sample_result, filepath in zip(decoder_result, file_paths):
        disp(sample_result, decoder_ix)

Decoder: 1
X min: 0, max: 253


In [11]:
!nvidia-smi

Sun Aug 11 07:22:47 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    32W /  70W |   1609MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [10]:
!ls

music-translation  sample_data	test.wav


In [2]:
%cd /content/music-translation

/content/music-translation


In [3]:
!python src/run_on_files.py --files notebooks/ --batch-size 1 --checkpoint checkpoints/pretrained_musicnet/lastmodel --output-next-to-orig --decoders "0" --py

Starting
xs size: torch.Size([1, 1, 80000])
torch.Size([1, 64, 100])
  0% 0/5 [00:00<?, ?it/s]
Generating:   0% 0/20 [00:00<?, ?it/s]/content/music-translation/src/wavenet_generator.py:104: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probabilities = F.softmax(prediction)

Generating:   5% 1/20 [00:21<06:43, 21.25s/it]
Generating:  10% 2/20 [00:42<06:23, 21.31s/it]
Generating:  15% 3/20 [01:03<06:01, 21.27s/it]
Generating:  20% 4/20 [01:25<05:40, 21.29s/it]
Generating:  25% 5/20 [01:46<05:19, 21.28s/it]
Generating:  30% 6/20 [02:07<04:57, 21.26s/it]
Generating:  35% 7/20 [02:28<04:36, 21.27s/it]
Generating:  40% 8/20 [02:50<04:16, 21.36s/it]
Generating:  45% 9/20 [03:11<03:54, 21.34s/it]
Generating:  50% 10/20 [03:33<03:33, 21.32s/it]
Generating:  55% 11/20 [03:54<03:12, 21.43s/it]
Generating:  60% 12/20 [04:16<02:51, 21.41s/it]
Generating:  65% 13/20 [04:37<02:30, 21.48s/it]
Generating:  70% 14/20 [04:59<02:

In [4]:
2180850 / 1000 / 60

36.3475

In [5]:
2226641 / 1000 / 60

37.110683333333334